### Data-driven optimization and decision making

### Final project

Juha Reinikainen

In [1]:
import pandas as pd
import numpy as np
import model
import data
import matplotlib.pyplot as plt

In [2]:
X = data.get_data()
print(X.shape)

results = []
for w in model.createWeightVectors(X.shape[1], 13):
    er = model.expected_return(X, w)
    r = model.risk(X, w)
    results.append((w, er, r))

print(max(results, key=lambda x: x[1]))
print(min(results, key=lambda x: x[2]))
print(len(results))
returns = list(map(lambda x: x[1], results))
risks = list(map(lambda x: x[2], results))
plt.scatter(risks, returns)
plt.show()


423.08029358193534
